In [1]:
   
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
#from tensorflow.python.framework import ops from cnn_utils import *

from math import pi
import itertools
import time

%matplotlib inline

#with tf.Session() as sess:
#    hello = tf.constant('Hello, TensorFlow!')
#    print(sess.run(hello))


/Users/hagar/apps/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:

np.random.seed(seed=123)

class Vertex:
    """a polar coordinate vertex of unit length"""
    def __init__(self, index):
        self.index = index
        if(index == 0):
            self.theta = tf.constant([0.0], dtype=tf.float64, name = "theta" + str(index))
            self.phi = tf.constant([0.0], dtype=tf.float64, name = "phi" + str(index))
        else:   
            self.theta = tf.get_variable("theta" + str(index), [1] , dtype=tf.float64, 
                                     initializer=tf.constant_initializer(np.random.random_sample() * pi))
            if(index == 1):
                self.phi = tf.constant([0.0], dtype=tf.float64, name = "phi" + str(index))
            else:
                self.phi = tf.get_variable("phi" + str(index), [1] , dtype=tf.float64,
                                   initializer=tf.constant_initializer(np.random.random_sample() * 2 * pi))
        #print( index )
        #print(self.theta)
        #print(self.phi)
        sin_theta = tf.sin(self.theta)
        self.x = tf.multiply(sin_theta, tf.cos(self.phi) , name='x' + str(self.index))
        self.y = tf.multiply(sin_theta, tf.sin(self.phi) , name='y' + str(self.index))
        self.z = tf.cos(self.theta, name='z' + str(self.index))
        self.xyz = tf.concat([self.x, self.y, self.z], axis=0)


def make_vertextes( n_vertexes ):
    print('make vertextes')
    vertexes = []
    for i in range(n_vertexes):
        vertexes.append( Vertex(i) )
    return vertexes

def make_pairs( vertexes ):
    print('make pairs')
    list = []
    for pair in itertools.combinations(vertexes, 2):
        list.append( 1 / tf.norm(pair[0].xyz-pair[1].xyz))
    return tf.add_n(list)    



In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
from sklearn import preprocessing

from pythreejs import *
import numpy as np
from IPython.display import display

def make_face_normals(verts,simplices):
    print(verts)
    print(simplices)
    face_normals = np.zeros((simplices.shape[0],3))
    for i in range(simplices.shape[0]):
        v = face_normals[i]
        for vertex_index in simplices[i]:
            #print(vertex_index)
            v += verts[vertex_index]
            
    face_normals = preprocessing.normalize(face_normals, norm='l2')   
    print(face_normals) 
    
    tuples=[]
    for i in range(face_normals.shape[0]):
        tuples.append((face_normals[i],i))
    return tuples    


def fix_simplex_normals(verts, simplices):
    #print('fix_simplex_normals')
    base = 1
    left = 0
    right = 2
    for face in simplices:
        vA = verts[ face[left] ];
        vB = verts[ face[base] ];
        vC = verts[ face[right] ];

        cb = vC - vB
        ab = vA - vB
        cross = np.cross( cb, ab )
        dot = np.dot( vA, cross)
        if dot < 0:
            #print( face )
            tmp  = face[left]
            face[left] = face[right]
            face[right] = tmp
            #print( dot )
            #print( face )
    return None

def make_vert_spheres(verts):
    spheres = []
    for v in verts:
        sphereGeom = SphereGeometry( radius = 0.03)
        sphereMesh = Mesh(geometry=sphereGeom, material = LambertMaterial(color='#444444',transparent=False, opacity=0.9))
        sphereMesh.position = v.tolist()
        spheres.append(sphereMesh)
    return spheres    


def makeMeshes(verts, simplices):
    #fix_simplex_normals(verts, simplices)
    fix_simplex_normals(verts, simplices)
    
    # https://github.com/jovyan/pythreejs
    cubeGeometry = PlainGeometry(vertices=verts, faces=simplices)   
    # , faceNormals=make_face_normals(pos,simplices)
    
    #,transparent=True, opacity=0.7, side='DoubleSide'
    myobjectCube = Mesh(geometry=cubeGeometry, material = LambertMaterial(color='#888888'))
    myobjectCube.material.wireframe = True
    camera = PerspectiveCamera(position=[-0.5, 6, -0.5], fov=25, aspect=1.0,
                          children=[DirectionalLight(color='#aaaaaa', position=[20, 20, 30], intensity=4)])
    spheres = make_vert_spheres(verts)
    sceneCube = Scene(children=[myobjectCube, AmbientLight(color='#aaaaaa', intensity=4)]+spheres)

    renderer = Renderer(camera=camera, background='black', background_opacity=1,
                            scene = sceneCube, controls=[OrbitControls(controlling=camera)], width='800',height='800')
   
    return renderer

In [4]:
import multiprocessing

n_vertexes = 16 # 12 -24 28 +32 36 40 -42 46

learning_rate = 6/(n_vertexes**2) 

start_time = time.time()

tf.reset_default_graph()
vertextes = make_vertextes(n_vertexes)
potential = make_pairs(vertextes)

multiplier = 1.02
learning_rate_placeholder = tf.placeholder(tf.float32, [], name='learning_rate')
print('make optimizer') 
optimizer = tf.train.GradientDescentOptimizer( learning_rate=learning_rate_placeholder ).minimize(potential)

#print([v.xyz for v in vertextes])
allPositions = tf.stack([v.xyz for v in vertextes])

NUM_THREADS = max(1, int( multiprocessing.cpu_count() / 2 ))
print('NUM_THREADS = ' + str(NUM_THREADS))
 
print('start session') 
#config = tf.ConfigProto(intra_op_parallelism_threads=4, inter_op_parallelism_threads=4, \
#                        allow_soft_placement=True, device_count = {'CPU': 4})
#config = tf.ConfigProto(intra_op_parallelism_threads=4, inter_op_parallelism_threads=4, \
#                        allow_soft_placement=True, device_count = {'CPU': 1})
#session = tf.Session(config=config)
session = tf.Session()

with session as sess:
    sess.run(tf.global_variables_initializer())
    last_p = 1e6
    last_was_equal = False
    for i in range(100000):
        if(i % 10 == 0):
            _ , p = sess.run([optimizer, potential], feed_dict={learning_rate_placeholder: learning_rate} )
            if(i % 200 == 0):
                print( str(i) + ' ' + "{0:.16f},  ".format(p)+ " ({0:.16e}),  ".format(last_p-p), end='')
            #if(i % 50 == 0):
            #    print( str(i) + ' ' + "{0:.8f}".format(p) + ' learning_rate = ' + 
            #          str( learning_rate) + ' multiplier = ' + str(multiplier) )
            #print( p - last_p )
            if abs(p - last_p) < 1e-11:
                if last_was_equal:
                    print('early stop')
                    break
                else:
                    last_was_equal = True
            else:
                last_was_equal = False
            #if( p < last_p ):
            #    learning_rate *= multiplier
            #else:
            #    learning_rate = max(1e-3, learning_rate/8)
            #    multiplier = 1 + (multiplier-1)/2
            last_p = p;
        else:
            sess.run( optimizer, feed_dict={learning_rate_placeholder: learning_rate} )  
    
    elapsed_time = time.time() - start_time
    print( str(elapsed_time)  + ' sec')
    print( str(i) + ' epochs' )
    
    verts = sess.run(allPositions, feed_dict={learning_rate_placeholder: learning_rate})    
    #print(verts)
    
    

make vertextes
make pairs
make optimizer
NUM_THREADS = 4
start session
0 114.9797388100429458,   (9.9988502026118990e+05),  200 93.0088116729456829,   (4.2522088676264502e-03),  400 92.9897186480053648,   (2.8508861788623108e-04),  600 92.9865378351394725,   (9.7218114618158324e-05),  800 92.9846558108236110,   (1.1372845767709805e-04),  1000 92.9804129373276425,   (3.6566041696062257e-04),  1200 92.9638387812165661,   (1.4264067334295305e-03),  1400 92.9284853439750265,   (1.3571139234755947e-03),  1600 92.9178772846032643,   (1.6402027006279241e-04),  1800 92.9156854629566169,   (8.8063351412870361e-05),  2000 92.9141912409857298,   (6.2429108538708533e-05),  2200 92.9132020139823851,   (3.8635566042444225e-05),  2400 92.9126157836431901,   (2.2114507544301887e-05),  2600 92.9122845065758156,   (1.2439377769624116e-05),  2800 92.9120961495338804,   (7.2075136756666325e-06),  3000 92.9119838864065457,   (4.4474378597669784e-06),  3200 92.9119119474894006,   (2.9677572399577912e-06),  

KeyboardInterrupt: 

In [ ]:

hull = ConvexHull(verts)

display(makeMeshes(verts, hull.simplices))

In [ ]:


def add_ordered(set, index1, index2):
    if index1 < index2:
        set.add((index1, index2))
    else:
        set.add((index2, index1))

def get_edges(simplexes):
    edges = set()
    for s in simplexes:
        add_ordered(edges, s[0], s[1])
        add_ordered(edges, s[1], s[2])
        add_ordered(edges, s[0], s[2])
    return list(edges)    

class EdgeGroup:
    """all edges that are the same distance"""
    def __init__(self, distance):
        self.distance = distance
        self.count = 1
        
    def __str__(self):
        return str(self.distance) + ':' + str(self.count)


def make_surface_edge_groups(verts, simplexes, epsilon):
    groups = []
    edges = get_edges(simplexes)
    for e in edges:
        distance = np.linalg.norm(verts[e[0]] - verts[e[1]])
        found = False
        for g in groups:
            if abs(g[0] - distance) < epsilon:
                g[1]+=1
                found=True
                break
        if not found:      
            groups.append( [distance,1] ) 
                                 
    return list(groups)
    
epsilon = 1e-5    
groups = make_surface_edge_groups(verts, hull.simplices, epsilon) 
print(groups)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.rcdefaults()
fig, ax = plt.subplots()

# Example data
people = [group[0] for group in groups]
y_pos = np.arange(len(people))
performance = [group[1] for group in groups]

ax.barh(y_pos, performance, align='center',
        color='darkblue', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(people)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Edge Count')
ax.set_title('Edge Length')

plt.show()

Todo:
1. fix face normals, https://github.com/mrdoob/three.js/blob/dev/src/core/Geometry.js computeFaceNormals
cubeGeometry.faceNormals

2. edge stats bar chart
4. transparent faces
5. edge color groups
5. manual change learning rate on split run on session
5. adaptive learning rate, exponetial average of last delta for each learning rate with N buckets randomly chosen each an increment more or less than best in the middle
5. save and load to file and use as init
5. graph
    1. e f v
    2. potential per ?
    3. groups
    4. periodic table of groups


## 12 Vertexes
![exampleOutput.png](exampleOutput.png)


## 32 Vertexes
![exampleOutput-32.png](exampleOutput-32.png)


In [ ]:
#cubeGeometry.faceNormals.__class__


In [ ]:
" {0:.16e},  ".format(1e-9)